In [1]:
from examples.seismic import Receiver, Model

from examples.seismic.acoustic import acoustic_setup
from examples.seismic.vector_reflectivity import vector_reflectivity_setup

import numpy as np

from devito import norm

In [2]:
nx = 200
nz = 120

shape = (nx, nz)

In [3]:
tn = 500.  # Final time

presets = {
    'layers-elastic': {'preset': 'layers-elastic', 'nlayers': 2},
    'layers-isotropic': {'preset': 'layers-isotropic', 'nlayers': 2},
}
# Create solver from preset
# solver = acoustic_setup(shape=(20, 25), spacing=[15. for _ in shape],
#                         nbl=10, tn=tn, space_order=2, time_order=1,
#                         **(presets['layers-isotropic']), dtype=np.float64)
solver = vector_reflectivity_setup(shape=(20, 25), spacing=[15. for _ in shape],
                        nbl=10, tn=tn, space_order=2, time_order=1,
                        **(presets['layers-isotropic']), dtype=np.float64, density=True)

# Create adjoint receiver symbol
srca = Receiver(name='srca', grid=solver.model.grid,
                time_range=solver.geometry.time_axis,
                coordinates=solver.geometry.src_positions)

# Run forward and adjoint operators
rec = solver.forward(save=False)[0]
solver.adjoint(rec=rec, srca=srca)

# Adjoint test: Verify <Ax,y> matches  <x, A^Ty> closely
term1 = np.dot(srca.data.reshape(-1), solver.geometry.src.data)
term2 = norm(rec) ** 2
print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'
    % (term1, term2, (term1 - term2)/term1, term1/term2))


assert np.isclose((term1 - term2)/term1, 0., atol=1.e-11)

Operator `initdamp` ran in 0.01 s
Operator `Forward` ran in 0.01 s
No source type defined, returning uninitiallized (zero) source
Operator `Adjoint` ran in 0.01 s
/tmp/ipykernel_19326/2762868582.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'


<x, A*y>: -1105494095007296645485617496210985580778167111113506816.000000, <Ax,y>: 10477719834958476.000000, difference: 1.0000e+00, ratio: -105509033684873078448465258994156437504.000000


AssertionError: 